# Тест по теме "Z-оценка. Выбросы".

In [15]:
import pandas as pd
import numpy as np
from scipy.stats import zscore

In [16]:
marvel_characters = pd.read_csv('Data/MarvelComicCaracters.csv', sep=';')

#### Функция по нахождению выбросов за границами основанными на полуторном интерквартильном размахе.

In [17]:

def outliers_iqr(original_dataframe, column_name, info='hide'):
    """Принимает датафрейм, название колонки и опциональный параметр вывода информации info=out.
    
    Откидывает пустые значения в заданной колонке.
    Рассчитывает количество выбросов из полуторного интерквартильного размаха в заданной колонке,
    выдаёт датафрейм с выбросами.
    """

    dataframe = original_dataframe.copy()
    # Избавляемся от пустых значений в рассматриваемой колонке
    dataframe = dataframe.dropna(subset=[column_name])
    median = dataframe[column_name].median()
    q1 = dataframe[column_name].quantile(0.25)
    q3 = dataframe[column_name].quantile(0.75)
    iqr = q3 - q1
    iqr_bound_botton = q1 - 1.5 * iqr
    iqr_bound_top = q3 + 1.5 * iqr
    
    outliers_dataframe = dataframe[(dataframe[column_name] > iqr_bound_top) | (dataframe[column_name] < iqr_bound_botton)]

    if info == 'out':
        print(column_name)
        print(f'Первый квартиль: {q1},'
              f'\nМедиана: {median},'
              f'\nТретий квартиль: {q3},'
              f'\nИнтерквартильный размах: {iqr},'
              f'\nВерхняя граница: {iqr_bound_top},'
              f'\nНижняя граница: {iqr_bound_botton}'
              )
        print(f'Количество выбросов по переменной {column_name}: {outliers_dataframe.shape[0]}\n')

    return outliers_dataframe

#### Функция по нахождению выбросов заданной переменной за границами трёх среднеквадратичных интервалов для переменных, представленных натуральными числами.

In [18]:
def outliers_std(original_dataframe, column_name, ddof=1, info='hide'):
    """Принимает датафрейм, название колонки, ddof и опциональный параметр вывода информации info=out. 
    
    Откидывает пустые значения в заданной колонке.
    Рассчитывает количество выбросов из трёх среднеквадратических отклонений в заданной колонке,
    выдаёт датафрейм с выбросами.
    """
    dataframe = original_dataframe.copy()
    # Избавляемся от пустых значений в рассматриваемой колонке
    dataframe = dataframe.dropna(subset=[column_name])
    mean = dataframe[column_name].mean()
    std = dataframe[column_name].std(ddof=ddof)
    # В зависимости от ddof сообщаем для чего рассчитывается среднеквадратическое
    # отклонение: для выборки, или для генеральной последовательности
    if ddof == 0:
        ddof_print = ' генеральной совокупности '
    else:
        ddof_print = ' выборки '
    botton = mean - 3 * std
    top = mean + 3 * std
    
    outliers_dataframe = dataframe[(dataframe[column_name] > top) | (dataframe[column_name] < botton)]

    if info == 'out':
        print(column_name)
        print(f'Среднее: {mean},\nСреднеквадратическое откл.{ddof_print}: {std},'
              f'\nВерхняя граница: {top},'
              f'\nНижняя граница: {botton}'
              )
        print(f'Количество выбросов по переменной {column_name}: {outliers_dataframe.shape[0]}\n')

    return outliers_dataframe

#### 1. Сколько пропущенных значений в переменной appearances? 

In [19]:
marvel_characters.isna().sum()['appearances']

1096

#### 2. Укажите, верно ли следующее утверждение: наибольший межквартильный размах по переменной appearances будет для злых персонажей. Вам понадобятся данные по типу персонажа (переменная align), удалите пропущенные значения по столбцу appearances и посчитайте требуемую меру разброса.

*Ответ: утверждение ложно*

In [20]:
marvel_characters_without_na_appearances = marvel_characters.dropna(subset=['appearances'])
q1 = marvel_characters_without_na_appearances.groupby('align', dropna=False)['appearances'].quantile(0.25)
q3 = marvel_characters_without_na_appearances.groupby('align', dropna=False)['appearances'].quantile(0.75)
iqr = q3 - q1
display(iqr)

align
Добрый персонаж         13.0
Злой персонаж            5.0
Нейтральный персонаж     8.0
NaN                      4.0
Name: appearances, dtype: float64

#### 3. Сколько выбросов встречается по переменной appearances? Работайте с исходными данными, предварительно удалите пропущенные значения по столбцу appearances, используйте для нахождения выбросов межквартильный размах.

In [21]:
outliers_iqr(marvel_characters, 'appearances', 'out').shape[0]

appearances
Первый квартиль: 1.0,
Медиана: 3.0,
Третий квартиль: 8.0,
Интерквартильный размах: 7.0,
Верхняя граница: 18.5,
Нижняя граница: -9.5
Количество выбросов по переменной appearances: 1938



1938

#### 4. На основе данных, полученных в третьем номере, укажите, верно ли следующее утверждение: наибольшее количество выбросов по переменной appearances наблюдается у добрых персонажей.

*Ответ: утверждение верно*

In [22]:
marvel_characters_without_na_appearances = marvel_characters.dropna(subset=['appearances'])
marvel_characters_without_na_appearances[
    # 18.5 - верхняя граница отсечения выбросов с помощью интерквартильного размаха,
    # определённая в задании №3. 
    # Нижняя граница не имеет физического смысла, поэтому смотрим выбросы
    # только за верхней границей.
    marvel_characters_without_na_appearances['appearances'] > 18.5
].groupby('align', dropna=False).count()['appearances']

align
Добрый персонаж         944
Злой персонаж           551
Нейтральный персонаж    285
NaN                     158
Name: appearances, dtype: int64

#### 5. Создайте новую переменную, которая будет представлять собой прологарифмированную переменную appearances.
##### Сколько выбросов получится по новой переменной? Работайте с исходными данными, предварительно удалите пропущенные значения по новому столбцу, используйте для нахождения выбросов три среднеквадратичных отклонения от среднего.

In [23]:
marvel_characters_log = marvel_characters.copy()
marvel_characters_log['log_appearances'] = np.log(marvel_characters_log['appearances'])
# Работаем с генеральной совокупностью, поэтому третьим аргументом в функции будет ddof=0
outliers_std(marvel_characters_log, 'log_appearances', ddof=0, info='out').shape[0]

log_appearances
Среднее: 1.341357279949188,
Среднеквадратическое откл. генеральной совокупности : 1.3444025706858918,
Верхняя граница: 5.374564992006863,
Нижняя граница: -2.6918504321084873
Количество выбросов по переменной log_appearances: 195



195

#### 6. Сколько выбросов получится по переменной appearances? Работайте с исходными данными, предварительно удалите пропущенные значения по столбцу appearances, используйте для нахождения выбросов три среднеквадратичных отклонения от среднего. Введите ответ в виде целого числа.

In [24]:
outliers_std(marvel_characters, 'appearances', ddof=0, info='out').shape[0]

appearances
Среднее: 17.033376963350786,
Среднеквадратическое откл. генеральной совокупности : 96.36980575831078,
Верхняя граница: 306.1427942382831,
Нижняя граница: -272.0760403115815
Количество выбросов по переменной appearances: 128



128

#### 7. Сколько выбросов получится по переменной appearances, если выбраны только женские персонажи? Работайте с исходными данными, выберите женских персонажей, удалите пропущенные значения по столбцу appearances, используйте для нахождения выбросов z-оценку и три среднеквадратичных отклонения от среднего.

In [25]:
girls = marvel_characters[marvel_characters['sex'] == 'Женский персонаж'].dropna(subset='appearances')
girls['z-score appearances'] = zscore(girls['appearances'])

girls[(girls['z-score appearances'] > 3) | (girls['z-score appearances'] < -3)].shape[0]

48

#### 8. Сравните среднее значение по переменной appearancs в двух датафреймах — без выбросов, определенных по межквартильному размаху, и без выбросов, определенных по трем среднеквадратичным отклонениям от среднего. Будем считать, что границы датафреймов без выбросов содержат значения верхних и нижних границ, определенных по межквартильному размаху или среднеквадратичному отклонению от среднего. В ответ запишите число (наибольшее среднее из двух), округлите до целого.

In [26]:
# Датафрейм без выбросов, определенный по трем среднеквадратичным отклонениям от среднего.
marvel_characters_without_outliers_std = marvel_characters[
    ~marvel_characters.isin(outliers_std(marvel_characters, 'appearances', ddof=0,))
# Очистим датафрейм от строк с пустым значением 'appearances', чтобы он выглядел чище.
# Это не влияет на результат функции mean(), так как она не учитывает пустые значения.
].dropna(subset='appearances')

print(
    round(
        marvel_characters_without_outliers_std['appearances'].mean()
    )
)

# Датафрейм без выбросов, определенный по межквартильному размаху.
# Находим датафрейм с выбросами через функцию и на его основе
# делаем инвертированную маску.
# Функция outliers_iqr относит к выбросам значения не включённые в границы,
# за счёт строгих неравенств.
marvel_characters_without_outliers_iqr = marvel_characters[
    ~marvel_characters.isin(outliers_iqr(marvel_characters, 'appearances'))
].dropna(subset='appearances')

print(
    round(
        marvel_characters_without_outliers_iqr['appearances'].mean()
    )
)

11
4


#### 9. Укажите, верно ли следующее утверждение: в датафрейме для добрых женских персонажей мода по переменной hair не изменится, если удалить выбросы по переменной appearances. Работайте с исходными данными, предварительно удалите пропущенные значения по столбцу appearances, используйте для нахождения выбросов межквартильный размах.

*Ответ: утверждение верно*

In [27]:
# Датафрейм с выбросами без пропущенных значений по столбцу 'appearances'
marvel_characters_with_outliers = marvel_characters.copy().dropna(subset='appearances')

# Сформируем маску для выборки женских добрых персонажей из датафрейма с выбросами.
good_girls = (
    (marvel_characters_with_outliers['sex'] == 'Женский персонаж')
    & (marvel_characters_with_outliers['align'] == 'Добрый персонаж')
)

# Датафрейм без выбросов, в рамках, определённых полуторным интерквартильным размахом.
# Находим датафрейм с выбросами через функцию и на его основе делаем инвертированную маску.
marvel_characters_in_iqr = marvel_characters[
    ~marvel_characters.isin(outliers_iqr(marvel_characters, 'appearances'))
].dropna(subset='appearances')

# Для чистоты вычислений найдём маску заново, но уже с учётом датафрейма без выбросов.
good_girls_no_na =  (
    (marvel_characters_in_iqr['sex'] == 'Женский персонаж')
    & (marvel_characters_in_iqr['align'] == 'Добрый персонаж')
)

# Сравним моды датафрейов с выбросами и без выбросов по модам переменной 'hair' у добрых женских персонажей
marvel_characters_with_outliers[good_girls]['hair'].mode() == marvel_characters_in_iqr[good_girls_no_na]['hair'].mode()

0    True
Name: hair, dtype: bool

#### 10. Выберите только злых персонажей. Сравните медиану по переменной appearances в двух датафреймах — с выбросами и без выбросов, определенных по межквартильному размаху. В ответ запишите число (наибольшее среднее из двух), округлите до целого.

In [28]:
# Датафрейм с выбросами
bad_guys = marvel_characters[marvel_characters['align'] == 'Злой персонаж']

# Медиана датафрейма с выбросами
print(bad_guys['appearances'].median())

# Датафрейм без выбросов
bad_guys_no_na = bad_guys[
    ~bad_guys.isin(outliers_iqr(bad_guys, 'appearances'))
]

# Медиана датафрейма без выбросов
print(bad_guys_no_na['appearances'].median())

3.0
2.0
